In [1]:
import sys
sys.path.append('/home/jjwhit/rcGAN/')

In [2]:
import torch
import yaml
import types
import json

import numpy as np
import matplotlib.patches as patches

from data.lightning.MassMappingDataModule import MMDataModule
from utils.parse_args import create_arg_parser
from pytorch_lightning import seed_everything
from models.lightning.mmGAN import mmGAN
from utils.mri.math import tensor_to_complex_np
import matplotlib.pyplot as plt
from matplotlib import gridspec
from scipy import ndimage

/home/jjwhit/rcGAN/fastmri/__init__.py:16: UserWarning: Could not retrieve fastmri version!
  warnings.warn("Could not retrieve fastmri version!")


In [3]:
# Import model here
test_plot_model = mmGAN.load_from_checkpoint('/share/gpu0/jjwhit/mass_map/mm_models/mmgan_training_ks/checkpoint-epoch=88.ckpt')

In [4]:
%run /home/jjwhit/rcGAN/scripts/mass_map/validate.py --config /home/jjwhit/rcGAN/configs/mass_map.yml --exp-name mmgan_training_ks --num-gpus 1

[rank: 0] Global seed set to 1
/home/jjwhit/.conda/envs/cGAN/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/jjwhit/.conda/envs/cGAN/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
)
VALIDATING EPOCH: 80


Computing generated distribution:   0%|          | 0/55 [00:00<?, ?it/s]/home/jjwhit/rcGAN/data/lightning/MassMappingDataModule.py:96: RuntimeWarning: divide by zero encountered in divide
  F_kappa = F_gamma / D
/home/jjwhit/rcGAN/data/lightning/MassMappingDataModule.py:96: RuntimeWarning: divide by zero encountered in divide
  F_kappa = F_gamma / D
/home/jjwhit/rcGAN/data/lightning/MassMappingDataModule.py:96: RuntimeWarning: divide by zero encountered in divide
  F_kappa = F_gamma / D
/home/jjwhit/rcGAN/data/lightning/MassMappingDataModule.py:96: RuntimeWarning: divide by zero encountered in divide
  F_kappa = F_gamma / D
Computing generated distribution:   0%|          | 0/55 [00:00<?, ?it/s]/home/jjwhit/rcGAN/data/lightning/MassMappingDataModule.py:96: RuntimeWarning: divide by zero encountered in divide
  F_kappa = F_gamma / D
/home/jjwhit/rcGAN/data/lightning/MassMappingDataModule.py:96: RuntimeWarning: divide by zero encountered in divide
  F_kappa = F_gamma / D
/home/jjwhit/rcG

IndexError: The shape of the mask [300, 300] at index 0 does not match the shape of the indexed tensor [495, 512] at index 0

In [ ]:
%run /home/jjwhit/rcGAN/scripts/mass_map/test.py --exp-name mmgan_training_8 --num-figs 10

In [5]:
import torch
import os
import yaml
import types
import json
import numpy as np

import sys
dir = '/home/jjwhit/rcGAN/'
sys.path.append(dir)

from data.lightning.MassMappingDataModule import MMDataModule
from utils.parse_args import create_arg_parser
from models.lightning.mmGAN import mmGAN
from pytorch_lightning import seed_everything
from utils.embeddings import VGG16Embedding
from evaluation_scripts.mass_map_cfid.cfid_metric import CFIDMetric

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [7]:
def load_object(dct):
    return types.SimpleNamespace(**dct)

In [8]:
torch.set_float32_matmul_precision('medium')
# seed_everything(1, workers=True)

with open(dir+'configs/mass_map.yml', 'r') as f:
    cfg = yaml.load(f, Loader=yaml.FullLoader)
    cfg = json.loads(json.dumps(cfg), object_hook=load_object)

In [9]:
dm = MMDataModule(cfg)
dm.setup()
val_loader = dm.val_dataloader()
best_epoch = -1
inception_embedding = VGG16Embedding()
best_cfid = 10000000
best_pearson = -1
best_psnr = -1
best_snr = -1
best_rmse = 10000000
start_epoch = 80  # Will start saving models after 80 epochs
end_epoch = 100
mask = np.load(
    "/home/jjwhit/rcGAN/mass_map_utils/cosmos/cosmos_mask.npy", allow_pickle=True
).astype(bool)

cfid_vals = []
psnr_vals = []
snr_vals = []
rmse_vals = []
r_vals = []

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
)


In [13]:
with torch.no_grad():
    
    for epoch in range(start_epoch, end_epoch):
        print(f"VALIDATING EPOCH: {epoch}")
        try:
            model = mmGAN.load_from_checkpoint(checkpoint_path='/share/gpu0/jjwhit/mass_map/mm_models/mmgan_training_ks' + f'/checkpoint-epoch={epoch}.ckpt')
        except Exception as e:
            print(e)
            continue

        if model.is_good_model == 0:
            print("NO GOOD: SKIPPING...")
            continue

        model = model.cuda()
        model.eval()

        cfid_metric = CFIDMetric(
            gan=model,
            loader=val_loader,
            image_embedding=inception_embedding,
            condition_embedding=inception_embedding,
            cuda=True,
            args=cfg,
            ref_loader=False,
            num_samps=1
        )

        recon, label, truth = cfid_metric._get_generated_distribution()

VALIDATING EPOCH: 80


Computing generated distribution:   0%|          | 0/55 [00:00<?, ?it/s]/home/jjwhit/rcGAN/data/lightning/MassMappingDataModule.py:96: RuntimeWarning: divide by zero encountered in divide
  F_kappa = F_gamma / D
/home/jjwhit/rcGAN/data/lightning/MassMappingDataModule.py:96: RuntimeWarning: divide by zero encountered in divide
  F_kappa = F_gamma / D
/home/jjwhit/rcGAN/data/lightning/MassMappingDataModule.py:96: RuntimeWarning: divide by zero encountered in divide
  F_kappa = F_gamma / D
/home/jjwhit/rcGAN/data/lightning/MassMappingDataModule.py:96: RuntimeWarning: divide by zero encountered in divide
  F_kappa = F_gamma / D
Computing generated distribution: 100%|██████████| 55/55 [00:22<00:00,  2.45it/s]


VALIDATING EPOCH: 81


KeyboardInterrupt: 

In [ ]:
# Testing CFID functions that the val script uses

In [54]:
import os
import yaml
import types
import json
import torch
import numpy as np
import sys
dir = '/home/jjwhit/rcGAN/'
sys.path.append(dir)
from data.lightning.MassMappingDataModule import MMDataModule
from utils.parse_args import create_arg_parser
from models.lightning.mmGAN import mmGAN
from pytorch_lightning import seed_everything
from utils.embeddings import VGG16Embedding
from evaluation_scripts.mass_map_cfid.cfid_metric import CFIDMetric
# import torchvision.transforms as transforms
from utils.mri.math import tensor_to_complex_np
from utils.mri.transforms import unnormalize_complex
from tqdm import tqdm
import torchvision.transforms as T

In [55]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [56]:
def load_object(dct):
    return types.SimpleNamespace(**dct)

In [57]:
torch.set_float32_matmul_precision('medium')
# seed_everything(1, workers=True)

with open(dir+'configs/mass_map_test.yml', 'r') as f:
    cfg = yaml.load(f, Loader=yaml.FullLoader)
    cfg = json.loads(json.dumps(cfg), object_hook=load_object)

In [58]:
dm = MMDataModule(cfg)
dm.setup()
val_loader = dm.val_dataloader()
best_epoch = -1
inception_embedding = VGG16Embedding()
best_cfid = 10000000
best_pearson = -1
best_psnr = -1
best_snr = -1
best_rmse = 10000000
start_epoch = 80  # Will start saving models after 80 epochs
end_epoch = 100
mask = np.load(
    "/home/jjwhit/rcGAN/mass_map_utils/cosmos/cosmos_mask.npy", allow_pickle=True
).astype(bool)

cfid_vals = []
psnr_vals = []
snr_vals = []
rmse_vals = []
r_vals = []

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
)


In [59]:
model = mmGAN.load_from_checkpoint(checkpoint_path=cfg.checkpoint_dir + 'mmgan_training_3/checkpoint-epoch=88.ckpt')
model = model.cuda()
model.eval()

mmGAN(
  (generator): UNetModel(
    (down_sample_layers): ModuleList(
      (0): ConvDownBlock(
        (conv_1): Conv2d(4, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (res): ResidualBlock(
          (conv_block): Sequential(
            (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): PReLU(num_parameters=1)
            (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (5): PReLU(num_parameters=1)
          )
          (conv_1x1): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1))
        )
        (conv_3): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation): PReLU(

In [14]:
def _get_generated_distribution(loader, gan, num_samps, args, cuda, ref_loader=None):
    image_embed = []
    cond_embed = []
    true_embed = []
    
    # Replace self.loader, self.gan, self.num_samps, etc., with parameters passed to this function
    for i, data in enumerate(loader):
        condition, gt, mean, std = data
        condition = condition.cuda()
        gt = gt.cuda()
        mean = mean.cuda()
        std = std.cuda()
        print('gt shape: ',gt.shape, 'condition shape: ',condition.shape, 'mean shape: ',mean.shape, 'std shape: ',std.shape)

        with torch.no_grad():
            for l in range(num_samps):
                recon = gan(condition)

                image = _get_embed_im(recon, args.kappa_mean, args.kappa_std)
                condition_im = _get_embed_im_complex(condition)
                true_im = _get_embed_im(gt, args.kappa_mean, args.kappa_std)

                img_e = image_embedding(transforms(image))
                cond_e = condition_embedding(transforms(condition_im))
                true_e = image_embedding(transforms(true_im))
                print('img_e shape: ',img_e.shape, 'cond_e shape: ',cond_e.shape, 'true_e shape: ',true_e.shape)

                if cuda:
                    true_embed.append(true_e)
                    image_embed.append(img_e)
                    cond_embed.append(cond_e)
                else:
                    true_embed.append(true_e.cpu().numpy())
                    image_embed.append(img_e.cpu().numpy())
                    cond_embed.append(cond_e.cpu().numpy())

    if ref_loader:
        with torch.no_grad():
            for i, data in enumerate(ref_loader):
                condition, gt, mean, std = data
                condition = condition.cuda()
                gt = gt.cuda()
                mean = mean.cuda()
                std = std.cuda()

                with torch.no_grad():
                    for l in range(num_samps):
                        recon = gan(condition)

                        image = _get_embed_im(recon, args.kappa_mean, args.kappa_std)
                        condition_im = _get_embed_im_complex(condition)
                        true_im = _get_embed_im(gt, args.kappa_mean, args.kappa_std)

                        img_e = image_embedding(transforms(image))
                        cond_e = condition_embedding(transforms(condition_im))
                        true_e = image_embedding(transforms(true_im))

                        if cuda:
                            true_embed.append(true_e)
                            image_embed.append(img_e)
                            cond_embed.append(cond_e)
                        else:
                            true_embed.append(true_e.cpu().numpy())
                            image_embed.append(img_e.cpu().numpy())
                            cond_embed.append(cond_e.cpu().numpy())
                        print('REF LOADER: img_e shape: ',img_e.shape, 'cond_e shape: ',cond_e.shape, 'true_e shape: ',true_e.shape)

    if cuda:
        true_embed = torch.cat(true_embed, dim=0)
        image_embed = torch.cat(image_embed, dim=0)
        cond_embed = torch.cat(cond_embed, dim=0)
    else:
        true_embed = np.concatenate(true_embed, axis=0)
        image_embed = np.concatenate(image_embed, axis=0)
        cond_embed = np.concatenate(cond_embed, axis=0)
    print('FINAL: image_embed shape: ',image_embed.shape, 'cond_embed shape: ',cond_embed.shape, 'true_embed shape: ',true_embed.shape)
    return image_embed.to(dtype=torch.float64), cond_embed.to(dtype=torch.float64), true_embed.to(dtype=torch.float64)
    
def _get_embed_im(multi_coil_inp, mean, std, im_size=300):
    # Initialize the tensor to store the embedded images
    embed_ims = torch.zeros(size=(multi_coil_inp.size(0), 3, im_size, im_size)).cuda()
    
    for i in range(multi_coil_inp.size(0)):
        # Create the reformatted tensor for each image in the batch
        reformatted = torch.zeros(size=(1, im_size, im_size, 2)).cuda()
        reformatted[:, :, :, 0] = multi_coil_inp[i, 0, :, :]
        reformatted[:, :, :, 1] = multi_coil_inp[i, 1, :, :]

        # Apply normalization based on mean and std
        unnormal_im = reformatted * std + mean

        # Convert to a complex numpy array and then to a real tensor
        im = torch.real(torch.tensor(tensor_to_complex_np(unnormal_im.cpu()))).cuda()

        # Normalize the image to the range [0, 1]
        im = (im - torch.min(im)) / (torch.max(im) - torch.min(im))

        # Assign the normalized image to the three channels of the output tensor
        embed_ims[i, 0, :, :] = im
        embed_ims[i, 1, :, :] = im
        embed_ims[i, 2, :, :] = im

    return embed_ims
    
def _get_embed_im_complex(multi_coil_inp, im_size=300):
    # Initialize the tensor to store the embedded images
    embed_ims = torch.zeros(size=(multi_coil_inp.size(0), 3, im_size, im_size)).cuda()

    for i in range(multi_coil_inp.size(0)):
        # Create the reformatted tensor for each image in the batch
        reformatted = torch.zeros(size=(1, im_size, im_size, 2)).cuda()
        reformatted[:, :, :, 0] = multi_coil_inp[i, 0, :, :]
        reformatted[:, :, :, 1] = multi_coil_inp[i, 1, :, :]

        # Unnormalize the complex image
        unnormal_im = unnormalize_complex(reformatted)  # Mean/std calculated during preprocessing

        # Convert to a complex numpy array and then to a real tensor
        im = torch.real(torch.tensor(tensor_to_complex_np(unnormal_im.cpu()))).cuda()

        # Normalize the image to the range [0, 1]
        im = (im - torch.min(im)) / (torch.max(im) - torch.min(im))

        # Assign the normalized image to the three channels of the output tensor
        embed_ims[i, 0, :, :] = im
        embed_ims[i, 1, :, :] = im
        embed_ims[i, 2, :, :] = im

    return embed_ims


In [15]:
image_embedding = VGG16Embedding()
condition_embedding = VGG16Embedding()
transforms = T.Compose([
    T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),])

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
)
Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
)


In [16]:
recon, label, truth = _get_generated_distribution(val_loader, model, 1, cfg, True, ref_loader=None)

gt shape:  torch.Size([9, 2, 300, 300]) condition shape:  torch.Size([9, 2, 300, 300]) mean shape:  torch.Size([9]) std shape:  torch.Size([9])
img_e shape:  torch.Size([9, 512]) cond_e shape:  torch.Size([9, 512]) true_e shape:  torch.Size([9, 512])
gt shape:  torch.Size([9, 2, 300, 300]) condition shape:  torch.Size([9, 2, 300, 300]) mean shape:  torch.Size([9]) std shape:  torch.Size([9])
img_e shape:  torch.Size([9, 512]) cond_e shape:  torch.Size([9, 512]) true_e shape:  torch.Size([9, 512])
gt shape:  torch.Size([9, 2, 300, 300]) condition shape:  torch.Size([9, 2, 300, 300]) mean shape:  torch.Size([9]) std shape:  torch.Size([9])
img_e shape:  torch.Size([9, 512]) cond_e shape:  torch.Size([9, 512]) true_e shape:  torch.Size([9, 512])
gt shape:  torch.Size([9, 2, 300, 300]) condition shape:  torch.Size([9, 2, 300, 300]) mean shape:  torch.Size([9]) std shape:  torch.Size([9])
img_e shape:  torch.Size([9, 512]) cond_e shape:  torch.Size([9, 512]) true_e shape:  torch.Size([9, 51

In [17]:
recon.shape

torch.Size([495, 512])

In [18]:
label.shape

torch.Size([495, 512])

In [19]:
truth.shape

torch.Size([495, 512])